In [3]:
!cp -r "/kaggle/input/vton-hd-main-new/VITON-HD-main" "/kaggle/working/"

In [4]:

import os

file_path = "/kaggle/working/VITON-HD-main/datasets.py"
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} deleted successfully.")
else:
    print("File does not exist.")



/kaggle/working/VITON-HD-main/datasets.py deleted successfully.


In [5]:
!cp -r "/kaggle/input/datasets-update3/datasets.py" "/kaggle/working/VITON-HD-main"


In [6]:
!pip install kornia


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.5 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [7]:
pip install torchgeometry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os

# Define new directory path
new_dir = "/kaggle/working/composites"

# Create the directory if it doesn't exist
os.makedirs(new_dir, exist_ok=True)

print(f"Directory created at: {new_dir}")


Directory created at: /kaggle/working/composites


In [9]:
!python /kaggle/working/VITON-HD-main/test.py \
    --name "my_kaggle_test" \
    --dataset_dir "/kaggle/input/clothes-tryon-dataset/clothes_tryon_dataset" \
    --checkpoint_dir "/kaggle/input/viton-repo2/VITON-HD-main/checkpoints" \
    --save_dir "./results" \
    --seg_checkpoint "seg_final.pth" \
    --gmm_checkpoint "gmm_final.pth" \
    --alias_checkpoint "alias_final.pth"

Namespace(name='my_kaggle_test', batch_size=1, workers=1, load_height=1024, load_width=768, shuffle=False, dataset_dir='/kaggle/input/clothes-tryon-dataset/clothes_tryon_dataset', dataset_mode='test', dataset_list='test_pairs.txt', checkpoint_dir='/kaggle/input/viton-repo2/VITON-HD-main/checkpoints', save_dir='./results', display_freq=1, seg_checkpoint='seg_final.pth', gmm_checkpoint='gmm_final.pth', alias_checkpoint='alias_final.pth', semantic_nc=13, init_type='xavier', init_variance=0.02, grid_size=5, norm_G='spectralaliasinstance', ngf=64, num_upsampling_layers='most')
Network [SegGenerator] was created. Total number of parameters: 34.5 million. To see the architecture, do print(network).
Network [ALIASGenerator] was created. Total number of parameters: 100.5 million. To see the architecture, do print(network).
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5015: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0.

In [31]:
import os
import random
from PIL import Image

# --- Configuration: UPDATE THESE PATHS ---
# 1. Path to your generated result images
results_dir = '/kaggle/working/results/my_kaggle_test'

# 2. Path to your original person photos
person_dir = '/kaggle/input/clothes-tryon-dataset/clothes_tryon_dataset/test/image'

# 3. Path to your original cloth images
cloth_dir = '/kaggle/input/clothes-tryon-dataset/clothes_tryon_dataset/test/cloth'

# 4. Where to save the final composite images
output_dir = '/kaggle/working/composites'

# 5. The height to standardize all images to
IMG_HEIGHT = 512
# ----------------------------------------

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

try:
    # Get all result images and select 10 random ones
    all_results = [f for f in os.listdir(results_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    if len(all_results) < 10:
        print(f"Warning: Found only {len(all_results)} images. Processing all of them.")
        selected_results = all_results
    else:
        selected_results = random.sample(all_results, 10)

    print(f"Processing {len(selected_results)} random images...")

    for result_filename in selected_results:
        try:
            # --- 1. NEW: Parse filename based on the latest screenshots ---
            filename_no_ext = os.path.splitext(result_filename)[0]
            parts = filename_no_ext.split('_')
            
            # Ensure the filename has the expected 3-part format (e.g., 00006_06396_00)
            if len(parts) < 3:
                print(f"  ⚠️  Skipping '{result_filename}': does not match expected format.")
                continue

            person_id_part = parts[0]
            cloth_id_part = parts[1]
            suffix_part = parts[2]

            # Construct the source filenames based on the new logic
            person_filename = f"{person_id_part}_{suffix_part}.jpg"
            cloth_filename = f"{cloth_id_part}_{suffix_part}.jpg"

            # --- 2. Find and open the three images ---
            person_img_path = os.path.join(person_dir, person_filename)
            cloth_img_path = os.path.join(cloth_dir, cloth_filename)
            result_img_path = os.path.join(results_dir, result_filename)

            person_img = Image.open(person_img_path)
            cloth_img = Image.open(cloth_img_path)
            result_img = Image.open(result_img_path)

            # --- 3. Standardize heights while maintaining aspect ratio ---
            def resize_to_height(img, height):
                aspect_ratio = img.width / img.height
                new_width = int(aspect_ratio * height)
                return img.resize((new_width, height), Image.Resampling.LANCZOS)

            person_img_resized = resize_to_height(person_img, IMG_HEIGHT)
            cloth_img_resized = resize_to_height(cloth_img, IMG_HEIGHT)
            result_img_resized = resize_to_height(result_img, IMG_HEIGHT)

            # --- 4. Create the composite image ---
            total_width = person_img_resized.width + cloth_img_resized.width + result_img_resized.width
            composite_img = Image.new('RGB', (total_width, IMG_HEIGHT), 'white')

            # Paste the images side-by-side
            composite_img.paste(person_img_resized, (0, 0))
            composite_img.paste(cloth_img_resized, (person_img_resized.width, 0))
            composite_img.paste(result_img_resized, (person_img_resized.width + cloth_img_resized.width, 0))

            # --- 5. Save the final image ---
            output_path = os.path.join(output_dir, f"composite_{result_filename}")
            composite_img.save(output_path)
            print(f"  ✅ Saved: {output_path}")

        except FileNotFoundError as e:
            print(f"  ❌ Error: Could not find a matching file for '{result_filename}'. Details: {e}")
        except Exception as e:
            print(f"  ❌ An unexpected error occurred for '{result_filename}': {e}")

except FileNotFoundError:
    print(f"Error: A directory was not found. Please check your paths and try again.")

Processing 10 random images...
  ✅ Saved: /kaggle/working/composites/composite_06768_01969_00.jpg
  ✅ Saved: /kaggle/working/composites/composite_12501_07965_00.jpg
  ✅ Saved: /kaggle/working/composites/composite_04269_12171_00.jpg
  ✅ Saved: /kaggle/working/composites/composite_02278_11237_00.jpg
  ✅ Saved: /kaggle/working/composites/composite_09500_06914_00.jpg
  ✅ Saved: /kaggle/working/composites/composite_07212_11330_00.jpg
  ✅ Saved: /kaggle/working/composites/composite_03429_07276_00.jpg
  ✅ Saved: /kaggle/working/composites/composite_07738_14009_00.jpg
  ✅ Saved: /kaggle/working/composites/composite_02268_02605_00.jpg
  ✅ Saved: /kaggle/working/composites/composite_04313_08032_00.jpg


In [32]:
import shutil
import os

# --- Configuration ---
# 1. Path to the folder you want to zip
folder_to_zip = '/kaggle/working/composites'

# 2. Desired name for the output zip file (without the .zip extension)
output_zip_name = 'composite_images'
# ---------------------

# Check if the folder exists
if not os.path.isdir(folder_to_zip):
    print(f"Error: Folder not found at '{folder_to_zip}'")
else:
    # Create the zip archive
    shutil.make_archive(output_zip_name, 'zip', folder_to_zip)
    print(f"✅ Successfully created '{output_zip_name}.zip'")

✅ Successfully created 'composite_images.zip'
